# Downloads e imports

In [1]:
import numpy as np
from sklearn.metrics import classification_report
from Models.LSTM.LSTM import LSTMModel
from busca_hiperparametros_NN import run_hiperparameter_search
import torch

import sys, os
sys.path.append('..')


RANDOM_SEED = 33

c:\Users\raafm\Pessoal\Ciencia-de-dados\Stock-Prices-Prediction\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
get_dataset_path = lambda  stock, get_labels,get_train: os.path.join("..","FinalDatasets", stock,f"{stock}_{'y' if get_labels else 'X'}_timeseries_{'train' if get_train else 'test'}.npy")

X_train = np.load(get_dataset_path(stock= 'VALE3', get_labels= False, get_train= True))
y_train = np.load(get_dataset_path(stock= 'VALE3', get_labels= True, get_train= True))
X_test  = np.load(get_dataset_path(stock= 'VALE3', get_labels= False, get_train= False))
y_test  = np.load(get_dataset_path(stock= 'VALE3', get_labels= True, get_train= False))

X_train.shape, X_test.shape

((370, 7, 62), (115, 7, 62))

# Busca por Hiperparâmetros

In [3]:
study = run_hiperparameter_search(X_train, y_train, ModelClass= LSTMModel, n_trials= 10)

[I 2024-07-11 03:41:17,370] A new study created in memory with name: no-name-36982b8a-9924-4984-a4e6-99db58573336
[I 2024-07-11 03:41:29,622] Trial 0 finished with value: 0.8189923240918067 and parameters: {'hidden_size': 88, 'num_layers': 3, 'learning_rate': 0.0007494643706838835}. Best is trial 0 with value: 0.8189923240918067.
[I 2024-07-11 03:41:36,983] Trial 1 finished with value: 0.8592432657204314 and parameters: {'hidden_size': 37, 'num_layers': 3, 'learning_rate': 0.0025320375766107327}. Best is trial 1 with value: 0.8592432657204314.
[I 2024-07-11 03:41:46,198] Trial 2 finished with value: 0.7733898282659892 and parameters: {'hidden_size': 127, 'num_layers': 5, 'learning_rate': 0.0001404166039589777}. Best is trial 1 with value: 0.8592432657204314.
[I 2024-07-11 03:41:52,376] Trial 3 finished with value: 0.8759108934718691 and parameters: {'hidden_size': 26, 'num_layers': 2, 'learning_rate': 0.003402079247794696}. Best is trial 3 with value: 0.8759108934718691.
[I 2024-07-11 

Number of finished trials:  10
Best trial:
  Value:  0.8759108934718691
  Params: 
    hidden_size: 26
    num_layers: 2
    learning_rate: 0.003402079247794696


# Treinando com os melhores Hiperparâmetros

In [4]:
model = LSTMModel(input_size= X_train.shape[-1],
                  hidden_size= study.best_params["hidden_size"], 
                  num_layers= study.best_params["num_layers"],
                  output_size= 1)

model.fit(X_train, y_train,
          learning_rate= study.best_params['learning_rate'],
          )

# Avaliando o Modelo

In [5]:
y_pred = model.predict(X_train)
print(classification_report(y_train,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       187
           1       1.00      1.00      1.00       183

    accuracy                           1.00       370
   macro avg       1.00      1.00      1.00       370
weighted avg       1.00      1.00      1.00       370



In [6]:
y_pred = model.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.61      0.71      0.65        68
           1       0.44      0.34      0.39        47

    accuracy                           0.56       115
   macro avg       0.53      0.52      0.52       115
weighted avg       0.54      0.56      0.54       115



In [7]:
print("Number of features:",X_train.shape[-1])
torch.save(model, os.path.join('Models','LSTM','all_features.pt'))

Number of features: 62
